<a href="https://colab.research.google.com/github/annefrancine/Deep-Learning-Models-/blob/master/Boltzmann_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MovieLens dataset**
https://grouplens.org/datasets/movielens/ 

# Importing Packages

In [0]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing Dataset


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
movies = pd.read_csv('/content/drive/My Drive/curso deep learning-udemy/Notebooks/7.BM/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')


In [29]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [30]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3883 non-null   int64 
 1   1       3883 non-null   object
 2   2       3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [31]:
users = pd.read_csv('/content/drive/My Drive/curso deep learning-udemy/Notebooks/7.BM/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [32]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       6040 non-null   int64 
 1   1       6040 non-null   object
 2   2       6040 non-null   int64 
 3   3       6040 non-null   int64 
 4   4       6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB


In [33]:
ratings = pd.read_csv('/content/drive/My Drive/curso deep learning-udemy/Notebooks/7.BM/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [34]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   0       1000209 non-null  int64
 1   1       1000209 non-null  int64
 2   2       1000209 non-null  int64
 3   3       1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


# Train and Test Dataset 

In [0]:

training_set = pd.read_csv('/content/drive/My Drive/curso deep learning-udemy/Notebooks/7.BM/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('/content/drive/My Drive/curso deep learning-udemy/Notebooks/7.BM/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [36]:
training_set.shape

(79999, 4)

In [37]:
test_set.shape

(19999, 4)

# Number of users and films

In [0]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

# Converting data into a matrix with users in rows and movies in columns

In [0]:
def convert(data):
    new_data = [] 
    for id_users in range(1, nb_users + 1): 
        id_movies = data[:,1][data[:,0] == id_users] 
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings

        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Creating Torch Tensors

In [0]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Converting the rating to binary : 1 like / 0 dislike

In [0]:

training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1  
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Creating the neural network architecture

In [0]:
class RBM():
    def __init__(self, nv, nh): 
        self.W  = torch.randn(nv, nh) 
        self.a = torch.randn(1, nh) 
        self.b = torch.randn(1, nv) 
    def sample_h(self, x): 
        wx = torch.mm(x, self.W) 
        activation = wx + self.a.expand_as(wx) 
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v) 
    def sample_v(self, y): 
        wy = torch.mm(y, self.W.t())  
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0) 
        self.a += torch.sum((ph0 - phk), 0) 
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

# Training neural network

In [48]:

nb_epoch = 10
for epoch in range(1, nb_epoch + 1): 
    train_loss = 0 
    s = 0. 
    #for id_user in range(0, nb_users - batch_size, batch_size): 
    for id_users in range(0,nb_users - batch_size, batch_size):
      vk = training_set[id_user:id_user+batch_size] 
      v0 = training_set[id_user:id_user+batch_size] 
      ph0,_ = rbm.sample_h(v0) 
      for k in range(10): 
          _,hk = rbm.sample_h(vk)
          _,vk = rbm.sample_v(hk)
          vk[v0<0] = v0[v0<0]
      phk,_ = rbm.sample_h(vk)
      rbm.train(v0, vk, ph0, phk)
      train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        #train_loss += torch.mean(torch.abs(v0[v0>0]  - vk[v0>=0]))
      s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s)) 

epoch: 1 loss: tensor(0.1957)
epoch: 2 loss: tensor(0.1602)
epoch: 3 loss: tensor(0.1546)
epoch: 4 loss: tensor(0.1533)
epoch: 5 loss: tensor(0.1479)
epoch: 6 loss: tensor(0.1524)
epoch: 7 loss: tensor(0.1474)
epoch: 8 loss: tensor(0.1504)
epoch: 9 loss: tensor(0.1458)
epoch: 10 loss: tensor(0.1488)


# Testing neural network

In [49]:

test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2413)
